In [1]:
import numpy as np
import holoviews as hv; hv.extension('bokeh', logo=False)
import panel as pn;     pn.extension()
from panel.interact import interact

import sympy as sp
from IPython.display import display, Latex, Math

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

def format_matrix_with_parentheses(A):
    A_latex = sp.latex(A)
    return A_latex.replace("\\begin{bmatrix}", "\\begin{pmatrix}").replace("\\end{bmatrix}", "\\end{pmatrix}")

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [2]:
%%julia
using Pkg, Revise
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, LaTeXStrings, Latexify, Markdown, SymPy, Random

  Activating project at `~/elementary-linear-algebra/GenLinAlgProblems`


<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Schur Decomposition: A Gateway to Eigenvalues and Stability</strong>
</div>    

# 1. Introduction

## 1.1 Schur's Lemma

The eigendecomposition of a square matrix $A$ of size $N \times N$ exists only if $A$ has a complete set<br>
of $N$ linearly independent eigenvectors.

One approach to address this even when the matrix is degenerate<br> (and actually the most useful in numerical analysis) is using a Schur decomposition:<br>
Instead of a **diagonal form**, this decomposition obtains an **upper triangular form** of the matrix $A$.<br>
Interestingly, this can be achieved with a set of orthonormal basis vectors.

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Schur's Lemma:** Any square matrix $A$ has the form $\;\; A = Q T Q^H,\;\;$
where
- $Q$ is unitary, i.e.,  $Q^H Q = I$
- $T$ is upper triangular
</div>

**Examples:**
* $A=\begin{pmatrix} 4 & \;\;1 \\ 2 & \;\;3\end{pmatrix}, \quad
Q = \frac{1}{\sqrt{2}} \left(\begin{array}{rr} 1 & -1 \\ 1 & 1 \end{array}\right), \quad T = \begin{pmatrix}5 & 1 \\ 0 & 2\end{pmatrix}\;\;$ satisfies $A = Q\ T\ Q^t$
* $A=\left(\begin{array}{rr} 1 & 1 \\ -2 & 3\end{array}\right), \quad
Q = \frac{1}{\sqrt{3}} \left(\begin{array}{rr}  1 & 1-i \\ 1+i & -1 \end{array}\right), \quad T = \begin{pmatrix} 2 + i & -1+2i \\ 0 & 2-i\end{pmatrix}\;\;$ satisfies $A = Q\ T\ Q^t$

## 1.2 A Constructive Proof

The key idea of Schur's Lemma is to **iteratively construct the triangular matrix** $T$<br> using
orthogonal matrices $𝑄_i$ while maintaining the similarity transform $A = Q\ T\ Q^t$

### 1.2.1 Use an Eigenvector to Introduce Zeros in the First Column

**Reminder:** Any $N \times N$ matrix $A$ has at least one eigenpair $(\lambda,x)$ for every distinct eigenvalue $\lambda$.<br><br>
Let us chose one such eigenpair, and without loss of generality assume that $x$ has unit length, i.e., $x^t x = 1$.<br><br>
Extend $\left\{ x \right\}$ to a full orthonormal basis of $N$ vectors. Note that since $A$ may have complex eigenvalues,<br>
$\qquad$ the resulting matrix
$Q = \begin{pmatrix} x &q_2&q_3&\dots &q_N\end{pmatrix} = \begin{pmatrix} x & \tilde{Q}\end{pmatrix}$
may have complex entries.

$\qquad \begin{aligned}
Q^H A Q &= \begin{pmatrix} x^H \\ \tilde{Q}^H \end{pmatrix}\  A\ \begin{pmatrix} x & &\tilde{Q}\end{pmatrix}\qquad & \\
&=         \begin{pmatrix} x^H \\ \tilde{Q}^H \end{pmatrix} \begin{pmatrix} A x & A \tilde{Q} \end{pmatrix} & \text{ now use } A x = \lambda x \;\; \text{ and } x^t x = 1 & \\
&=         \begin{pmatrix} \lambda & x^H A \tilde{Q}^H \\
                           \lambda \tilde{Q}^H x & \tilde{Q}^H A \tilde{Q}\end{pmatrix} &
                           \text{ but } x \perp q_2, q_3, \dots q_n \\ 
&=          \begin{pmatrix} \lambda & x^H A \tilde{Q}^H \\
                           0 & \tilde{Q}^H A \tilde{Q}\end{pmatrix} &
\end{aligned}$

Observe that $\tilde{a}^H = x^H A \tilde{Q}^H$ is a row vector, and $\;\;\tilde{A} = \tilde{Q}^H A \tilde{Q}\;\;$ is a matrix of size $(N-1) \times (N-1)$, i.e.,

$\qquad Q^H A Q = \begin{pmatrix} \lambda & \tilde{a}^H \\ 0 & \tilde{A} \end{pmatrix}$.

### 1.2.2 Repeat with $\tilde{A}$

To see that we can continue this process with successively smaller matrices $\tilde{A}$:
* Let $Q_1^H A Q_1 = \begin{pmatrix} \lambda_1 & \tilde{a_1}^H \\ 0 & \tilde{A}_1 \end{pmatrix}$
* obtain $\;\;\tilde{Q}_2^H \tilde{A}_1 \tilde{Q}_2 = \begin{pmatrix} \lambda_2 & \tilde{a}_2^H \\ 0 & \tilde{A}_2 \end{pmatrix}$ for some given eigenpair $(\lambda_2, x_2)$ of $\tilde{A}_1$
* Set $Q_2 = \begin{pmatrix} 1 & 0 \\ 0 & \tilde{Q_2}\end{pmatrix}\;\;$ and therefore
$\;\;(Q_2 Q_1)^H A (Q_1 Q_2) = \begin{pmatrix} \lambda_1 & \dots & \dots \\
                0          & \lambda_2 & \dots \\
                0          & 0 & \tilde{A}_3 \end{pmatrix}$
* repeat this process for each matrix $\tilde{A}_i,\;\;$ resulting in an upper triangular matrix<br>
  $\;\;T = (Q_N \dots Q_2 Q_1)^H A (Q_1 Q_2) =
  \begin{pmatrix} \lambda_1 & \dots     & \dots  &  \dots  &  \dots \\
                 0          & \lambda_2 & \dots  &  \dots  &  \dots \\
                 0          & 0         & \ddots &  \dots  &  \dots \\
                 0          & 0         &  0     & \ddots  &  \dots \\
                 0          & 0         &  0     & \dots & \lambda_N
  \end{pmatrix}$

In [3]:
%%julia
"""
given a vector v, obtain a unitary matrix Q with v̂ as its first column
"""
function naive_unitary_matrix_from_vector(v)
    """
    Given a vector `v`, augment it with the identity matrix and return the Q matrix from QR decomposition.
    Parameters:
        v::Vector: The input vector.
    Returns:
        Matrix: Unitary matrix obtained from QR decomposition.
    """
    n = length(v)
    augmented_matrix = hcat(v, 1I(n)) # Augment vector with identity matrix
    Q, _ = qr(augmented_matrix) # Perform QR decomposition
    return Matrix(Q) # Convert Q to a standard matrix
end
;

In [4]:
%%julia
"""
given a matrix A, obtain a Schur triangularization
"""
function naive_schur_triangularization(A::AbstractMatrix)
    n, m = size(A)
    @assert n == m "Matrix A must be square"

    Q = one(eltype(A)) * I(n)

    for i in 1:n-1
        subA = A[i:end, i:end]                       # Extract the current submatrix

        λ, V = eigen(subA)                           # Compute the eigenvectors
        v = V[:, 1]                                  # First eigenvector

        U_sub = naive_unitary_matrix_from_vector(v)        # Construct a unitary matrix using v
        U     = Matrix( one(eltype(U_sub)) * I(n))   # Full-size identity matrix
        U[i:end, i:end] = U_sub                      # Embed the submatrix into U
        py_show( L"\text{Step }", i, L":\quad \tilde{A} =", subA, L", \quad v_%$i =", round.(v,digits=2),  L", \quad Q_%$i =", round.(U,digits=2), inline=true)

        A = U' * A * U                               # Apply the unitary transformation '
        Q = Q * U                                    # Update Q
    end
    return Q, A                                      # T = A
end
;

In [5]:
%%julia
A   = [1 3 0 ; -3 1 0; -2 4 0 ]
py_show(L"\text{Triangularize } A =", A, color="blue", inline=true)
Q, T = naive_schur_triangularization(A)
py_show( L"A = Q T Q^H, \quad Q =", round.(Q,digits=3), L", \quad T = ", round.(T, digits=3), color="blue", inline=true)
@show A ≈ Q*T*Q';

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

A ≈ Q * T * Q' = true

## 1.3 Eigenvalues and Eigenvectors

**Reminder:** For unitary matrices $Q$, the matrices $A$ and $T$ related by $A = Q T Q^H$ have the same eigenvalues.

Let $(\lambda, x)$ be an eigenpair of $A$, and let $A = Q T Q^h$ be a similarity transform of $A$.

$\qquad\begin{aligned}
p(\lambda) &= \det\left(A - \lambda I\right) \\
&= \det\left( Q T Q^H - \lambda Q Q^H \right)\\
&= \det\left( Q (T - \lambda I) Q^H \right) \\
&= \det\left( T - \lambda I \right)
\end{aligned}$

Further, for the eigenvector $x$, we have

$\qquad\begin{aligned}
A x = \lambda x & \Leftrightarrow\;\;  Q^H A x &= \lambda Q^H x \\
                & \Leftrightarrow\;\;  Q^H Q T Q^H x &= \lambda Q^H x \\
                & \Leftrightarrow\;\;  T Q^H x &= \lambda Q^H x \\
\end{aligned}$

Since $\Vert Q^H x \Vert = \Vert x \Vert$ we are guaranteed that $Q^H x \ne 0$, and thus
$\quad$ $\mathbf{\tilde{x} = Q^H x}\;\;$ **is an eigenvector of $T$**

In summary

* **Eigenvalues**
  - The eigenvalues of $A$ are the diagonal entries of $T$.
  - For example, if $T = \begin{pmatrix} \color{red}{2} & 5                 & 3 \\
                                          0             & \color{red}{2+3i} & 2i \\
                                          0             & 0                 & \color{red}{2-3i}
                          \end{pmatrix},\;\;$
then $A$ has eigenvalues $2, 2+3i$ and $2-3i$.
  - If $A$ is real with complex eigenvalues, they appear as **conjugate pairs** on the diagonal of $T$.

* **Eigenvectors**
    - The columns of $U$ form an **orthonormal basis** for the space.
    - If $A$ is diagonalizable, the columns of $U$ are its eigenvectors.
    - If $A$ is not diagonalizable, the columns of $Q$ still provide a basis aligned with the triangular structure of $T$,<br>
though they may not be eigenvectors.

* **Summary**
    - The Schur factorization reveals the eigenvalues (on the diagonal of $T$ ).
    - It provides an orthonormal basis (columns of $Q$ for $A$.
    - Even for non-diagonalizable matrices, $T$ is triangular, and $Q$ gives a stable basis for computations.

# 3. The QR Algorithm

## 3.1 Simplest Form of the Algorithm

In practice, we compute a Schur decomposition of a given matrix $A$ using the **QR algorithm:**

The method alternates between two steps
1. **QR Factorization**: Decompose the matrix $A$ into $A = Q R,\;\;$ where
   - $Q$ is a unitary matrix
   - $R$ is an upper triangular matrix
2. **Similarity Transformation**: Compute the similarity transform $Q^t A Q = Q^t Q R Q = Q R$<br>
   and repeat the process.

The algorithm is observed to iteratively transform the matrix $A$ into a triangular form.

In [6]:
%%julia
"""
Naive implementation of the QR algorithm to compute the Schur decomposition.

Parameters:
    A::AbstractMatrix: The input square matrix to decompose.
    max_iter::Int: Maximum number of iterations to run the algorithm (default: 1000).
    tol::Float64: Convergence tolerance for off-diagonal elements (default: 1e-10).

Returns:
    Q::Matrix: The unitary matrix from the Schur decomposition (A = Q * T * Q').
    T::Matrix: The upper triangular matrix from the Schur decomposition.
"""
function naive_qr_algorithm(A::AbstractMatrix; max_iter::Int=1000, tol::Float64=1e-10)
    # Ensure the matrix is square
    n, m = size(A)
    @assert n == m "Matrix A must be square"

    Q_total = I(n)
    Aₖ       = copy(A)
    conv     = Inf

    for k in 1:max_iter
        Qₖ, Rₖ   = qr(Aₖ)           # Perform QR decomposition
        Aₖ       = Rₖ * Qₖ          # Compute the next Ak
        Q_total *= Qₖ              # Accumulate Q

        conv    = norm(Aₖ - triu(Aₖ), Inf)  # Check for convergence (off-diagonal elements close to zero)
        if conv < tol break end
    end

    return Q_total, Aₖ, conv
end
;

In [7]:
%%julia
A          = [ 10.  2   4   4; 2 15   5   6; 3  5  20   7; 4  6   7  25 ]
Q, T, conv = naive_qr_algorithm(A)

py_show(L"A = ", Int.(A), L",\qquad \text{convergence} =", conv, inline=true, color="blue")
py_show(L"Q = ", round.(Q, digits=3), L", \quad T = ", round.(T, digits=3), inline=true)
@show A ≈ Q * T * Q';

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


A ≈ Q * T * Q' = true


____
This first attempt suffers from a number of shortcomings:
* the algorithm can stagnate or converge very slowly
* when the matrix is nonsymmetric, it can have complex eigenvalues.<br>
Complex eigenvalues require careful handling: the algorithm may fail to capture the complex structure<br>
and oscillate instead of converging.
* numerical errors may accumulate, leading to failure.

## 3.2 Improvement: First Reduce the Matrix to Hessenberg Form

### 3.2.1 The Hessenberg Form

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Definition:** A **Hessenberg matrix** is a square matrix where all elements below the first subdiagonal are zero.<br>
- For an **upper Hessenberg matrix**, the entries $h_{i j} = 0$ for all $i > j+1$.
- For a **lower Hessenberg matrix**, the entries $h_{i j} = 0$ for all $i < j-1$.
</div>

For example, an upper Hessenberg matrix has the form:

$\qquad
H = 
\begin{pmatrix}
\color{red}{h_{11}} & h_{12} & h_{13} & \cdots  & h_{1n} \\
h_{21} & \color{red}{h_{22}} & h_{23} & \cdots  & h_{2n} \\
0      & h_{32} & \color{red}{h_{33}} & \cdots  & h_{3n} \\
\vdots & \vdots & \ddots & \color{red}{\ddots}  & \vdots \\
 0      & 0      & \cdots & h_{n n-1} & \color{red}{h_{nn}}
\end{pmatrix}.
$

**Remarks:**
* A matrix can readily be put into Hessenberg form, e.g., using [**HouseholderReflections.ipynb**](HouseholderReflections.ipynb): $A = Q H$.
* Using this matrix $Q$ in a similarity transform $\tilde{H} = Q^t A Q$ maintains the Hessenberg form

**Idea: Use Hessenberg Form as input to the QR algorithm**

The QR algorithm iteratively factorizes a matrix $A$ into $A = QR$,<br>  
where $Q$ is orthogonal and $R$ is upper triangular.<br>
Using a Hessenberg matrix as input significantly enhances efficiency and stability:

1. **Lower Computational Cost**: For a general $n \times n$ matrix, QR factorization requires $O(n^3)$ operations.<br>  
   Reducing $A$ to Hessenberg form decreases this cost to $O(n^2)$, leveraging the sparsity below the subdiagonal.<br>

2. **Eigenvalue Preservation**: The transformation to Hessenberg form is a similarity transformation,<br>
ensuring $A$ and its Hessenberg form share the same eigenvalues.

3. **Convergence to Schur Form**: Successive QR iterations on the Hessenberg matrix $H_k$<br>
drive it toward an upper triangular (or quasi-upper triangular) form,<br>
representing the Schur decomposition of $A$.

Starting with a Hessenberg matrix streamlines the QR algorithm, combining computational efficiency<br>
with numerical stability for reliable eigenvalue and Schur decomposition calculations.

**Complex Eigenvalues:**

For a real square matrix $A$, the QR algorithm iteratively transforms the matrix into a quasi-upper triangular form as follows:

* If all eigenvalues are real, $A_k$ (the matrix after the $k^{th}$ iteration converges to an upper triangular matrix <br>
with eigenvalues on the diagonal.
* If $A$ has complex eigenvalues, the algorithm produces a block diagonal structure.<br>
  $2\times 2$ blocks appear along the diagonal corresponding to pairs of complex-conjugate eigenvalues.<br>
  The remaining diagonal entries are real and correspond to real eigenvalues.

##### Algorithm and Display Implementation

In [8]:
%%julia
function generate_matrix(n, seed=42)
    Random.seed!(seed)
    return rand(n, n)
end

function successive_qr_with_eigenvalue_tracking(A, num_iter=10)
    n = size(A, 1)
    matrices = [A]
    eigenvalue_evolution = []  # To store eigenvalue estimates for each iteration

    # Perform QR iterations
    for _ in 1:num_iter
        Q, R = qr(matrices[end])
        next_matrix = R * Q
        push!(matrices, next_matrix)

        # Extract eigenvalue estimates
        current_eigenvalues = []
        i = 1
        while i <= n
            if i < n && abs(next_matrix[i+1, i]) > 1e-12  # Detect a 2x2 block
                # Extract the 2x2 block
                a = next_matrix[i, i]
                b = next_matrix[i, i+1]
                c = next_matrix[i+1, i]
                d = next_matrix[i+1, i+1]

                # Compute eigenvalues of the 2x2 block
                trace = a + d
                det = a * d - b * c
                discriminant = trace^2 - 4 * det
                if discriminant >= 0
                    # Real eigenvalues
                    λ1 = (trace + sqrt(discriminant)) / 2
                    λ2 = (trace - sqrt(discriminant)) / 2
                else
                    # Complex eigenvalues
                    real_part = trace / 2
                    imag_part = sqrt(-discriminant) / 2
                    λ1 = real_part + im * imag_part
                    λ2 = real_part - im * imag_part
                end
                append!(current_eigenvalues, [λ1, λ2])
                i += 2  # Skip the next row as it's part of the block
            else
                # Single real eigenvalue
                push!(current_eigenvalues, next_matrix[i, i])
                i += 1
            end
        end
        push!(eigenvalue_evolution, current_eigenvalues)
    end

    return matrices, eigenvalue_evolution
end

# ================================================================================================
N              = 20   # matrix size
Num_iterations = 30

A              = generate_matrix(N)
Hq,H           = hessenberg(A)
H              = Matrix(H)

qr_A, evals_A  = successive_qr_with_eigenvalue_tracking(A, Num_iterations)
qr_H, evals_H  = successive_qr_with_eigenvalue_tracking(H, Num_iterations)
;

In [9]:
# Convert Julia arrays to Python
qr_A      = [np.array(mat) for mat in Main.qr_A]
qr_H      = [np.array(mat) for mat in Main.qr_H]
A         = np.array(Main.A)
H         = np.array(Main.H)
evals_A   = [np.array(eig) for eig in Main.evals_A]
evals_H   = [np.array(eig) for eig in Main.evals_H]

In [15]:
# Interactive Threshold Adjustment
threshold_slider = pn.widgets.FloatSlider(name="Threshold", start=0, end=1, step=0.01, value=0.01)
iteration_slider = pn.widgets.IntSlider(name="Iteration", start=0, end=len(qr_A) - 1, step=1, value=0)

def raster(matrix, title, threshold=None):
    if threshold is not None:
        # Apply threshold: Highlight values above the threshold as 1, else 0
        matrix = np.where(np.abs(matrix) > threshold, 1, 0)  # Binary mask
        cmap = "binary"  # Black and white colormap
    else:
        cmap = "Gray_r"  # Inverted grayscale colormap
    return hv.Raster(matrix).opts(
        cmap=cmap,  # Colormap for the visualization
        xaxis=None,  # Hide the x-axis
        yaxis=None,  # Hide the y-axis
        frame_width=250,  # Set frame width
        aspect="equal",  # Maintain square aspect ratio
        title=title,  # Add a title to the plot
    )

# Adjusted Raster Function
def raster_with_threshold(matrix, title):
    threshold = threshold_slider.value
    return raster(matrix, title, threshold)


@pn.depends(threshold_slider.param.value, iteration_slider.param.value)
def update_matrix_plots(threshold, iteration):
    """
    Update both the QR matrix and Hessenberg matrix plots dynamically.
    """
    qr_matrix = qr_A[iteration]  # Update QR matrix for the current iteration
    hessenberg_matrix = qr_H[iteration]  # Update Hessenberg matrix for the current iteration

    qr_plot = raster(qr_matrix, f"A_{iteration}", threshold=threshold)
    hessenberg_plot = raster(hessenberg_matrix, f"H_{iteration}", threshold=threshold)

    return pn.Row(qr_plot, hessenberg_plot)

# Combine into a Panel Column with Sliders
matrix_interactive = pn.Column(
    "### Interactive Matrix Structures",
    pn.Row(threshold_slider, iteration_slider),
    update_matrix_plots
)

def eigenvalue_complex_plot(eigenvalue_evolution_A, eigenvalue_evolution_H, title):
    num_iterations  = len(eigenvalue_evolution_A)
    num_eigenvalues = len(eigenvalue_evolution_A[0])

    # Slider for selecting the eigenvalue index
    slider = pn.widgets.IntSlider(name="Eigenvalue Index", start=0, end=num_eigenvalues - 1, step=1, value=0)

    def plot_eigenvalue(index):
        # Extract real and imaginary parts for the eigenvalue at `index`
        real_vals_A = np.array([eig[index].real for eig in eigenvalue_evolution_A])
        imag_vals_A = np.array([eig[index].imag for eig in eigenvalue_evolution_A])
        real_vals_H = np.array([eig[index].real for eig in eigenvalue_evolution_H])
        imag_vals_H = np.array([eig[index].imag for eig in eigenvalue_evolution_H])

        # Real Part Plot
        real_curve_A = hv.Curve(
            (np.arange(num_iterations), real_vals_A),
            "Iteration", "Real Part", label="Matrix A"
        ).opts(color="blue", line_width=2)

        real_curve_H = hv.Curve(
            (np.arange(num_iterations), real_vals_H),
            "Iteration", "Real Part", label="Matrix H"
        ).opts(color="green", line_width=2)

        real_overlay = (real_curve_A * real_curve_H).opts(
            title="Real Part",
            frame_width=300, frame_height=180, legend_position="top",
            framewise=True, axiswise=True
        )

        # Imaginary Part Plot
        imag_curve_A = hv.Curve(
            (np.arange(num_iterations), imag_vals_A),
            "Iteration", "Imaginary Part", label="Matrix A"
        ).opts(color="blue", line_width=2)

        imag_curve_H = hv.Curve(
            (np.arange(num_iterations), imag_vals_H),
            "Iteration", "Imaginary Part", label="Matrix H"
        ).opts(color="green", line_width=2)

        imag_overlay = (imag_curve_A * imag_curve_H).opts(
            title="Imaginary Part",
            frame_width=300, frame_height=180, legend_position="top",
            framewise=True, axiswise=True
        )

        # Combine the two plots into a vertical layout
        return real_overlay + imag_overlay

    # Use DynamicMap for interactivity
    dmap = hv.DynamicMap(lambda i: plot_eigenvalue(i), kdims=["Index"])
    dmap = dmap.redim.values(Index=list(range(num_eigenvalues))).opts(framewise=True, axiswise=True)

    # Combine slider and plots in a vertical layout
    return pn.Column(slider, pn.bind(lambda i: dmap.select(Index=i), slider))

# Matrix Structures Tab with Sliders
matrix_tabs = pn.Column(
    matrix_interactive,  # Add the interactive matrix plot with sliders
)


#qr_plots = (
#    qr_evolution_dynamic(qr_A, "QR on Original Matrix") +\
#    qr_evolution_dynamic(qr_H, "QR on Hessenberg Matrix")
#).cols(2)

#eigenvalue_tabs = pn.Tabs(
#    eigenvalue_complex_plot(evals_A, evals_H, "Eigenvalues Comparison")
#)
# Combine into Tabs-based Dashboard
dashboard_tabs = pn.Tabs(
    ("Matrix Structures",       matrix_tabs),
    #("QR Algorithm Evolution",  qr_plots),
    ("Eigenvalue Evolution",     eigenvalue_complex_plot(evals_A, evals_H, "Eigenvalues Comparison")),
)

#### Example Behavior

The following graphs investigate the behavior of the algorithm, comparing results for matrix $A$ and its Hessenberg form $H$
* **Matrix Structure** investigate the decay of the values in the lower triangular part, with values below threshold displayed in white.
* **Eigenvalue evolution** tracks the diagonal entries in the matrix: later iterates should approximate the actual eigenvalues

In [16]:
dashboard_tabs

Tabs
    [0] Column
        [0] Column
            [0] Markdown(str)
            [1] Row
                [0] FloatSlider(name='Threshold', step=0.01, value=0.01)
                [1] IntSlider(end=30, name='Iteration')
            [2] ParamFunction(function, _pane=Row, defer_load=False)
    [1] Column
        [0] IntSlider(end=19, name='Eigenvalue Index')
        [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

# 4. Take Away

## 4.1 Summary
- The Schur decomposition expresses a square matrix $A$ as $A = Q T Q^H$.
- The diagonal of $T$ contains the eigenvalues of $A$.
- $Q$ is unitary, preserving numerical stability.

#### Why Use Schur Decomposition?

While eigendecomposition also expresses a matrix in terms of its eigenvalues and eigenvectors, it has some limitations:
1. **Existence**:
   - Eigendecomposition exists only for diagonalizable matrices. Non-diagonalizable matrices, such as those with defective eigenvalues, cannot be decomposed using eigendecomposition.
   - Schur decomposition, however, exists for every square matrix, regardless of diagonalizability.

2. **Stability**:
   - Schur decomposition involves unitary matrices, which are numerically stable because they preserve lengths and angles during computations. This is especially important in applications requiring high precision.

3. **Practicality**:
   - The Schur decomposition retains more structure in the triangular matrix \( T \) compared to eigendecomposition, where the matrix is diagonal. This makes it a versatile tool in numerical linear algebra, particularly in iterative algorithms like QR or spectral analysis.

## 4.2 Applications of Schur Decomposition

The Schur decomposition is fundamental in various areas, including:
- **Eigenvalue Computation**: Eigenvalues of $A$ are readily found as the diagonal elements of $T$.
- **Control Theory**: Stability of a system can be analyzed by examining the eigenvalues of $A$ through its Schur form.
- **Spectral Analysis**: The decomposition provides a basis for understanding the spectrum of a matrix.
- **Matrix Functions**: Facilitates the computation of matrix functions, e.g., exponentials and logarithms.